In [44]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
import re

train = pd.read_csv(r"C:\Users\raven\Downloads\sf-crime\train.csv\train.csv")
test  = pd.read_csv(r"C:\Users\raven\Downloads\sf-crime\test.csv\test.csv")

train1 = train.drop(['Descript', 'Resolution'], axis = 1)

train1 = train1.query('Y != 90')

In [45]:
train1.head()

,Dates,Category,DayOfWeek,PdDistrict,Address,X,Y
0,2015-05-13 23:53:00,WARRANTS,Wednesday,NORTHERN,OAK ST / LAGUNA ST,-122.425892,37.774599
1,2015-05-13 23:53:00,OTHER OFFENSES,Wednesday,NORTHERN,OAK ST / LAGUNA ST,-122.425892,37.774599
2,2015-05-13 23:33:00,OTHER OFFENSES,Wednesday,NORTHERN,VANNESS AV / GREENWICH ST,-122.424363,37.800414
3,2015-05-13 23:30:00,LARCENY/THEFT,Wednesday,NORTHERN,1500 Block of LOMBARD ST,-122.426995,37.800873
4,2015-05-13 23:30:00,LARCENY/THEFT,Wednesday,PARK,100 Block of BRODERICK ST,-122.438738,37.771541


In [46]:
Dt = pd.to_datetime(train1['Dates'])
train1['Date'] = Dt.dt.date
train1['Year'] = Dt.dt.year
train1['Month'] = Dt.dt.month
train1['Day'] = Dt.dt.day
train1['Hour'] = Dt.dt.hour
train1.drop(["Dates"], axis = 1, inplace = True)

# le = LabelEncoder()
# train1['DayOfWeek'] = le.fit_transform(train1['DayOfWeek'])

# count_encoding = train1['PdDistrict'].value_counts().to_dict()
# train1['PdDistrict'] = train1['PdDistrict'].map(count_encoding)




train1['Category'] = le.fit_transform(train1['Category'])

train1.head()
# # DayOfWeek...label, PdDistrict...count

,Category,DayOfWeek,PdDistrict,Address,X,Y,Date,Year,Month,Day,Hour
0,37,Wednesday,NORTHERN,OAK ST / LAGUNA ST,-122.425892,37.774599,2015-05-13,2015,5,13,23
1,21,Wednesday,NORTHERN,OAK ST / LAGUNA ST,-122.425892,37.774599,2015-05-13,2015,5,13,23
2,21,Wednesday,NORTHERN,VANNESS AV / GREENWICH ST,-122.424363,37.800414,2015-05-13,2015,5,13,23
3,16,Wednesday,NORTHERN,1500 Block of LOMBARD ST,-122.426995,37.800873,2015-05-13,2015,5,13,23
4,16,Wednesday,PARK,100 Block of BRODERICK ST,-122.438738,37.771541,2015-05-13,2015,5,13,23


In [47]:
Dt = pd.to_datetime(test['Dates'])
test['Date'] = Dt.dt.date
test['Year'] = Dt.dt.year
test['Month'] = Dt.dt.month
test['Day'] = Dt.dt.day
test['Hour'] = Dt.dt.hour
test.drop(["Dates"], axis = 1, inplace = True)

# le = LabelEncoder()
# test['DayOfWeek'] = le.fit_transform(test['DayOfWeek'])

# count_encoding = test['PdDistrict'].value_counts().to_dict()
# test['PdDistrict'] = test['PdDistrict'].map(count_encoding)

# Ad_dict = {'.* / .*':0, '.*':1}
# test['Address'] = test['Address'].replace(Ad_dict,regex = True)
test.head()

,Id,DayOfWeek,PdDistrict,Address,X,Y,Date,Year,Month,Day,Hour
0,0,Sunday,BAYVIEW,2000 Block of THOMAS AV,-122.399588,37.735051,2015-05-10,2015,5,10,23
1,1,Sunday,BAYVIEW,3RD ST / REVERE AV,-122.391523,37.732432,2015-05-10,2015,5,10,23
2,2,Sunday,NORTHERN,2000 Block of GOUGH ST,-122.426002,37.792212,2015-05-10,2015,5,10,23
3,3,Sunday,INGLESIDE,4700 Block of MISSION ST,-122.437394,37.721412,2015-05-10,2015,5,10,23
4,4,Sunday,INGLESIDE,4700 Block of MISSION ST,-122.437394,37.721412,2015-05-10,2015,5,10,23


In [48]:
cat_features = ['DayOfWeek', 'PdDistrict', 'Address']

In [49]:
cols = ['DayOfWeek', 'PdDistrict', 'Address', 'X', 'Y', 'Year', 'Month', 'Day', 'Hour']

X = train1[cols]
Y = train1['Category']

X_test = test[cols]

from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X, Y, test_size = 0.3, random_state = 1219)

In [50]:
from catboost import CatBoostClassifier
cb = CatBoostClassifier()

cb.fit(X_train, y_train, cat_features = cat_features)

Learning rate set to 0.109784
0:	learn: 3.3221557	total: 49.6s	remaining: 13h 46m 20s
1:	learn: 3.1596855	total: 1m 44s	remaining: 14h 32m 30s
2:	learn: 3.0483844	total: 2m 44s	remaining: 15h 10m 16s


CatBoostError: bad allocation

In [ ]:
y_pred = cb.predict(X_val)

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(y_val, y_pred)

0.28691888608363864

In [ ]:
cb_submission = cb.predict(X_test)

In [ ]:
import csv
pred = cb.predict_proba(test[['DayOfWeek', 'PdDistrict', 'Address', 'X', 'Y', 'Year', 'Month', 'Day', 'Hour']])
submission = pd.DataFrame(columns=['Id'], data=test)
submission = pd.concat([submission, pd.DataFrame(pred, columns=["ARSON", "ASSAULT", "BAD CHECKS", "BRIBERY", "BURGLARY", "DISORDERLY CONDUCT",
"DRIVING UNDER THE INFLUENCE", "DRUG/NARCOTIC", "DRUNKENNESS", "EMBEZZLEMENT", "EXTORTION",
"FAMILY OFFENSES", "FORGERY/COUNTERFEITING", "FRAUD", "GAMBLING", "KIDNAPPING", "LARCENY/THEFT",
"LIQUOR LAWS", "LOITERING", "MISSING PERSON", "NON-CRIMINAL", "OTHER OFFENSES", "PORNOGRAPHY/OBSCENE MAT",
"PROSTITUTION", "RECOVERED VEHICLE", "ROBBERY", "RUNAWAY", "SECONDARY CODES", "SEX OFFENSES FORCIBLE",
"SEX OFFENSES NON FORCIBLE", "STOLEN PROPERTY", "SUICIDE", "SUSPICIOUS OCC", "TREA", "TRESPASS",
"VANDALISM", "VEHICLE THEFT", "WARRANTS", "WEAPON LAWS"])], axis=1)

In [ ]:
submission.to_csv("cat1.csv", index=False)